In [358]:
import pandas as pd

data = pd.DataFrame()
vocabs = []
with open('./vocab.txt', 'r') as f:
    for line in f:
        vocabs.append(line.split()[0])

rows = []
count = []
with open('./ap.dat', 'r') as f:
    for line in f:
        tmp = line.split(' ')
        count.append(tmp[0])
        tmp = tmp[1:]
        row = [0] * len(vocabs)
        for elem in tmp:
            index, value = elem.split(':')
            row[int(index)] = int(value)
        rows.append(row)
            
            
            
data = pd.DataFrame(rows, columns = vocabs)
data

,i,new,percent,people,year,two,million,president,last,government,...,closes,huts,privileges,adverse,elliot,clyde,ski,leaf,spilled,buffs
0,1,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,1,0,3,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,0,0,3,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,3,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2242,0,10,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2243,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2244,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [359]:
from gensim.corpora import Dictionary
from gensim.matutils import Sparse2Corpus
from gensim.models import LdaModel
import scipy.sparse as sp
K = 5
# Assume that your DataFrame is called 'df' and has the following columns: 'word1', 'word2', ..., 'wordN'
# where 'word1' to 'wordN' are the words and their corresponding counts in each document

# Create a dictionary from the DataFrame

bow_words = data.columns.tolist()

# Create a dictionary from the list of words
dictionary = Dictionary([bow_words])

dictionary.filter_tokens([bow_words.index(word) for word in dictionary.values() if word not in bow_words])

# Convert the DataFrame to a bag-of-words matrix
bow_matrix = data.values
bow_sparse = sp.csc_matrix(bow_matrix)
corpus = Sparse2Corpus(bow_sparse)
corpus.dictionary = dictionary




In [367]:
from gensim.models import CoherenceModel

min_topics = 2
max_topics = 10
coherence_scores = []
for num_topics in range(min_topics, max_topics+1):
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
    coherence_model = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
    coherence_scores.append((num_topics, coherence_model.get_coherence()))

# Find the number of topics with the highest coherence score
best_num_topics = max(coherence_scores, key=lambda x: x[1])[0]
print(f"The best number of topics is {best_num_topics}")


The best number of topics is 4


In [368]:
coherence_scores

[(2, -2.539125862812347),
 (3, -2.630705670682047),
 (4, -2.492166243999541),
 (5, -2.715827685051838),
 (6, -2.5352140132943894),
 (7, -2.626229482244856),
 (8, -2.625112706554312),
 (9, -2.6551391556767623),
 (10, -2.629098418308557)]

In [369]:
K = 4

# Train an LDA model on the corpus
lda_model = LdaModel(corpus, num_topics=K, id2word=dictionary)

# Get the topic distributions for all documents in the corpus
doc_topic_dists = lda_model[corpus]

# Print the top words for each topic
for i, topic in lda_model.show_topics(num_topics=K, num_words=10, formatted=False):
    print(f'Topic {i}:')
    print([word for word, _ in topic])

Topic 0:
['builders', 'april', 'consent', 'aspen', 'ap', 'chiles', 'broadcast', 'bells', 'alfred', 'couldnt']
Topic 1:
['anthony', 'battered', 'connections', 'cisneros', 'bizarre', 'capacity', 'asleep', 'atwater', 'arabs', 'channel']
Topic 2:
['confident', 'borer', 'absence', 'authentic', 'bourgeois', 'bus', 'category', 'contracted', 'audiences', 'class']
Topic 3:
['backlog', 'acquiring', 'chicago', 'computerized', 'clerical', 'abm', 'antimissile', 'complaining', 'ball', 'climate']


In [370]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Assume that 'lda_model' is the LDA model trained on your corpus

# Convert the gensim corpus to a format compatible with pyLDAvis
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization in a web browser
pyLDAvis.display(vis_data)
